### 1. 웹스크래핑 연습문제 (질문1, 질문2 통합)
* Daum 뉴스기사 제목 스크래핑하기 

In [3]:
import requests
import bs4
from bs4 import BeautifulSoup

#section_dict = {100:'정치',101:'경제',102:'사회',103:'생활/문화',104:'세계',105:'IT/과학'}
section_dict = {'기후/환경':'climate','사회':'society','경제':'economy','정치':'politics',
                '국제':'world','문화':'culture','생활':'life','IT/과학':'tech','인물':'people'}

def print_news(section_name):  #print_new(103,'생활/문화') 
  sid = section_dict.get(section_name,'society')
  url = f'https://news.daum.net/{sid}'
  print(f'{section_name} 뉴스 {url}')
  req_header = {
    'user-agent' : 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/144.0.0.0 Safari/537.36'
}
  res = requests.get(url, headers=req_header)
  res.encoding = 'utf-8'

  if res.ok:
    soup = BeautifulSoup(res.text,'html.parser')

    # CSS 선택자를 사용해서 a tag 목록 가져오기
    li_tags = soup.select('ul.list_newsheadline2 li')

    for li_tag in li_tags:
        a_tag = li_tag.find('a')
        link = a_tag['href']
        print(link)

        strong_tag = li_tag.select_one('div.cont_thumb strong.tit_txt')

        if strong_tag:
                    title = strong_tag.text.strip()
                    print(f'제목: {title}')
                    print(f'링크: {link}')
                    print('-' * 30)
        
  else :
  # 응답(response)이 Error 이면 status code 출력    
    print(f'Error Code = {res.status_code}')

In [ ]:
print_news("인물")

### 2. 웹 스크래핑 연습문제
* 2-1. Nate 뉴스기사 제목 스크래핑하기

In [14]:
import requests
import bs4
from bs4 import BeautifulSoup
from urllib.parse import urljoin # 1) 경로 결합을 위한 urljoin
from IPython.display import Image, display # 2) 이미지 출력을 위한 Image, display

# 섹션별 경로와 mid 값 정의
section_dict = {
    '최신': ('recent', 'n0100'),
    '정치': ('section', 'n0200'),
    '경제': ('section', 'n0300'),
    '사회': ('section', 'n0400'),
    '생활/문화': ('section', 'n0500'),
    'IT/과학': ('section', 'n0600')
}

def print_news(section_name):
    # 딕셔너리 정보 추출 및 URL 생성
    path, mid = section_dict.get(section_name, ('recent', 'n0100'))
    url = f'https://news.nate.com/{path}?mid={mid}'
    print(f'### {section_name} 뉴스 목록: {url} ###\n')
    
    req_header = {
        'user-agent' : 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/144.0.0.0 Safari/537.36'
    }
    
    res = requests.get(url, headers=req_header)
    res.encoding = res.apparent_encoding

    if res.ok:
        soup = BeautifulSoup(res.text, 'html.parser')

        # [해결포인트] 보내주신 HTML 구조인 a.lt1를 직접 찾거나, 일반적인 li 구조를 모두 찾습니다.
        # 선택자가 하나라도 맞으면 데이터를 가져옵니다.
        news_items = soup.select('a.lt1, .mlist2 li, .mduSubjectList li')

        if not news_items:
            print("데이터를 찾지 못했습니다. 선택자를 확인해주세요.")
            return

        for item in news_items:
            # 1) a 태그(기사 링크) 및 제목 추출
            # item 자체가 a 태그일 수도 있고(a.lt1), li일 수도 있으므로 분기 처리
            if item.name == 'a':
                a_tag = item
            else:
                a_tag = item.select_one('a')
            
            if not a_tag: continue

            # urljoin을 사용해 기사 링크 생성
            link = urljoin(url, a_tag['href'])
            
            # 제목 추출 (보내주신 h2.tit 포함)
            title_tag = item.select_one('h2.tit, strong.tit, span.tb, .tit')
            if not title_tag: continue
            
            title = title_tag.text.strip()

            # 3) img 엘리먼트 존재 여부 체크
            img_tag = item.select_one('img')

            # 출력 순서: Image -> 제목 -> 링크
            if img_tag and img_tag.get('src'):
                src = img_tag['src']
                # 1) urljoin으로 // 경로와 도메인 결합
                full_img_url = urljoin(url, src)
                if '///' in src:
                    src = src.split('///')[-1]
                
                # 2) Image 클래스와 display 함수로 이미지 출력
                display(Image(url=full_img_url, width=150))
            else:
                # 이미지가 없는 뉴스일 경우
                print("(이미지 없음)")
            
            print(f'제목: {title}')
            print(f'링크: {link}')
            print('-' * 60)
            
    else:
        print(f'Error Code = {res.status_code}')



In [ ]:
print_news('경제')

### 2-2. 하나의 네이버 웹툰과 1개의 회차에 대한 Image 다운로드 하기 (필수)

In [ ]:
import requests
from bs4 import BeautifulSoup
import os

webtoon_url = 'https://comic.naver.com/webtoon/detail?titleId=846643&no=4&week=tue'


req_header = {"referer": "https://comic.naver.com/webtoon/detail?titleId=846643&no=4&week=tue",
              'user-agent' : 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/144.0.0.0 Safari/537.36'}

def download_one_episode(title, no, url):
    # 1. 저장 디렉토리 설정 (img\제목\회차)
    # os.path.join을 사용하여 운영체제에 맞는 경로(img/제목/회차)를 생성합니다.
    imgdir_name = os.path.join('img', title, str(no))
    
    # 디렉토리가 없으면 하위 폴더까지 포함하여 생성 (exist_ok=True는 이미 폴더가 있어도 에러를 내지 않습니다)
    if not os.path.exists(imgdir_name):
        os.makedirs(imgdir_name, exist_ok=True)
        print(f"디렉토리 생성: {imgdir_name}")

res = requests.get(webtoon_url, headers=req_header)
if res.ok:
    # .jpg 파일명 추출해서 list에 저장하기
    # img_url_list = [] #list()
    soup = BeautifulSoup(res.text,'html.parser')
    print(len(soup.select("img[src*='IMAG01']")))
    img_tags = soup.select("img[src*='IMAG01']")
    # for img_tag in img_tags:
    #     img_url = img_tag['src']
    #     img_url_list.append(img_url)
    # print(img_url_list)

  # List Comprehension Pthonic (쉽게짠거)
    img_url_list = [img_tag['src'] for img_tag in img_tags]
    print(img_url_list)
    imgdir_name = 'img'
    if not os.path.isdir(imgdir_name):
        os.mkdir(imgdir_name) 
    for img_url in img_url_list:
        res = requests.get(img_url, headers=req_header)
        print(res.status_code)
        # binary 응답 데이터 가져오기
        img_data = res.content
        # url에서 파일명만 추출하기
        #img/xxxIMGA01.jpg
        file_path = os.path.join(imgdir_name,os.path.basename(img_url))
        # binday data를 file에 write하기
        with open(file_path, 'wb') as file:
            print(f'Writing to {file_path}({len(img_data):,}bytes)')
            file.write(img_data)

else :
  # 응답(response)이 Error 이면 status code 출력    
    print(f'Error Code = {res.status_code}')